<a href="https://colab.research.google.com/github/BiancaStadl/ProjektarbeitML/blob/main/Embedding_Glove_Vergleich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GloVe embeddings taken from:

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]

hatespeechdata taken from:
 Wiegand, M., Siegel, M. and Ruppenhofer, J., 2018. Overview of the GermEval 2018 Shared Task on the Identification of Offensive Language. In: Proceedings of GermEval 2018, 14th Conference on Natural Language Processing (KONVENS 2018). Vienna, Austria: Research Gate. available on: https://github.com/uds-lsv/GermEval-2018-Data (last checked: 09.05.2021)

In [1]:
#import matplotlib.pyplot as plt -> für evtl Visualisierungen
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import losses 
from keras import optimizers 
from keras import metrics 

#!pip install Tokenizer
from keras.preprocessing.text import Tokenizer
#!pip install pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [2]:

max_length = 60

In [3]:
url = "https://github.com/uds-lsv/GermEval-2018-Data/archive/master.zip"

dataset = tf.keras.utils.get_file("GermEval-2018-Data-master.zip", url, 
                                   extract=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'GermEval-2018-Data-master')


6635520/Unknown - 1s 0us/step

In [4]:
os.listdir(dataset_dir)

['results.pdf',
 'evaluationScriptGermeval2018.pl',
 'README.md',
 'germeval2018.test.txt',
 'LICENSE.md',
 'survey.pdf',
 'submissions.tgz',
 'germeval2018.training.txt',
 'guidelines-iggsa-shared.pdf']

In [5]:
training_file = os.path.join(dataset_dir, 'germeval2018.training.txt')

testing_file = os.path.join(dataset_dir, 'germeval2018.test.txt')


In [6]:
def remove_clutter(string):
   string = re.sub("@[^\s]+"," ",string)
   string = re.sub("#[^\s]+"," ", string)
   string = re.sub("\u00a9"," ", string)
   string = re.sub("\u00ae"," ", string)
   string = re.sub("[\u2000-\u3300]"," ", string)
   string = re.sub("\ud83c[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83d[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83e[\ud000-\udfff]"," ", string)
   string = re.sub("😜", " ",string)
   string = re.sub("🍫", " ",string)
   string = re.sub("😁", " ",string)
   string = re.sub("🐖", " ",string)
   string = re.sub("😡", " ",string)
   string = re.sub("😇", " ",string)
   string = re.sub("😬", " ",string)
   string = re.sub("😃", " ",string)
   string = re.sub("😂", " ",string)
   string = re.sub("💙", " ",string)  
   string = re.sub("😛", " ",string)
   string = re.sub("🙏", " ",string)
   string = re.sub("👍", " ",string)
   string = re.sub("🖕", " ",string)
   string = re.sub("😉", " ",string)
   string = re.sub("💩", " ",string)
   string = re.sub("🤢", " ",string)
   string = re.sub("👏", " ",string)
   string = re.sub("😨", " ",string)
   string = re.sub("🤣", " ",string)
   string = re.sub("🤡", " ",string)
   string = re.sub("😈", " ",string)
   string = re.sub("💃🏽", " ",string)
   string = re.sub("👹", " ",string)
   string = re.sub("🤘", " ",string)
   string = re.sub("😱", " ",string)
   string = re.sub("🤔", " ",string) 
   string = re.sub("🌈", " ",string) 
   string = re.sub("💕", " ",string) 
   string = re.sub("👩‍❤️‍👩", " ",string) 
   string = re.sub("😍", " ",string) 
   string = re.sub("👆", " ",string) 
   string = re.sub("😖", " ",string) 
   string = re.sub("👇", " ",string) 
   string = re.sub("🔥", " ",string) 
   string = re.sub("😘", " ",string) 
   string = re.sub("🎉", " ",string) 
   string = re.sub("🤬", " ",string) 
   string = re.sub("👊", " ",string)
   string = re.sub("🇩🇪", " ",string)  
   
   string = re.sub("OTHER|OFFENSE|ABUSE|INSULT"," ",string)
   return string

In [7]:
statementsForTraining = []
sentimentsForTraining = []


fileToRead = open(training_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  findSentiment = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)


  statementsForTraining.append(line)
   #sentimentsForTraining.append(findSentiment.group(0))

  if findSentiment.group(0) == "OTHER":  
    sentimentsForTraining.append(0)
  else:
    sentimentsForTraining.append(1)

  if not line:
    break
 
fileToRead.close()

training_sentences = statementsForTraining
training_labels = sentimentsForTraining

#print(training_sentences[9])
#print(training_labels[9])

#print(len(training_sentences))

In [8]:
##do the same with testdata
statementsForTesting = []
sentimentsForTesting = []

fileToRead = open(testing_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  sent = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)

    

  statementsForTesting.append(line)
  #print(len(line))
  #sentimentsForTesting.append(sent.group(0))

  if sent.group(0) == "OTHER": 
    sentimentsForTesting.append(0)
  else:
    sentimentsForTesting.append(1)

  if not line:
    break


fileToRead.close()


testing_sentences = statementsForTesting
testing_labels = sentimentsForTesting
print(len(testing_sentences))
#print(testing_sentences)   
#print(testing_labels)

3532


In [9]:
tokenizer = Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(training_sentences)

#creating a word index - nur die Trainigsdaten
word_index = tokenizer.word_index

validation_size = 500


training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_training = pad_sequences(training_sequences, maxlen=max_length, padding='post')
print(len(padded_training))

validation_sequences = padded_training[0:validation_size]
validation_labels = training_labels[0:validation_size]

padded_training = padded_training[validation_size:]
training_labels = training_labels[validation_size:]

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_testing = pad_sequences(testing_sequences, maxlen=max_length,  padding='post')

#print(validation_sequences[499])
print(padded_training[0])
#print(len(validation_labels))
#print(len(training_labels))


nppadded_training = np.array(padded_training)
nptraining_labels = np.array(training_labels)

nppadded_validation = np.array(validation_sequences)
npvalidation_labels = np.array(validation_labels)

nppadded_testing = np.array(padded_testing)
nptesting_labels = np.array(testing_labels)


print(len(nppadded_training))
print(len(nptraining_labels))
print(len(word_index))

#print(statementsForTraining[2])
#print(nppadded_training[4])
#print(nppadded_training.shape)
#print(nptraining_labels[4])
#print(nppadded_testing.shape)
#print(word_index) 

5009
[  12 3993   11   41 6730 1042    4    5  202   39    3 2930   49 1363
  812  495 3994    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
4509
4509
15130


In [10]:
from google.colab import drive
drive.mount("/content/drive")
os.listdir("/content/drive/MyDrive/Colab Notebooks")


Mounted at /content/drive


['germeval_training.txt',
 'glove.twitter.27B.50d.txt',
 'glove.twitter.27B.200d.txt',
 'glove.6B.200d.txt',
 'glove.840B.300d.txt',
 'tensorboard.gdoc',
 'keras.gdoc',
 'Vectorization CNN embedded_limited_vocab.ipynb',
 'LSTM_limited_vocab.ipynb',
 'LSTM.ipynb',
 'CNN embedded_limited_vocab.ipynb',
 'CNN embedded.ipynb',
 'glove.42B.300d.txt',
 'Embedding Glove Vergleich.ipynb']

First Embedding with Glove Twitter 200d

In [11]:
vocabulary_size = len(word_index)+2

In [12]:
#Größe Vokabel -> wordindex + 2 (weil padding + OOV) 
hits200 = 0
misses200 = 0



# dann erstell ich ein Wörterbuch mit Namen "embedding_vector", dort sind dann
#die keys drinnen, die in glove-Datei drinnen sind mit dem entsprechenden Key

embedding_index_glove = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.twitter.27B.200d.txt')
for line in f:
  value = line.split()
  word = value[0]
  coef = np.asarray(value[1:],dtype='float32')
  embedding_index_glove[word] = coef

print("%d gefunden: "% len(embedding_index_glove))

#Dann noch eine Embedding-Matrix erstellen
#zweiter Wert = Embedding-Dimension der Datei, in dem Fall 200

glove_matrix = np.zeros((vocabulary_size,200))
for word, index in tokenizer.word_index.items():
    embedding_value = embedding_index_glove.get(word)
    if embedding_value is not None:
      glove_matrix[index] = embedding_value
      hits200+=1
    else:
      misses200+=1

print("hits %d and %d misses"%(hits200,misses200))



1193514 gefunden: 
hits 6747 and 8383 misses


Second embedding try with glove.840B.300d.txt

In [15]:
hits300 = 0
misses300 = 0

embedding_index_glove300 = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.42B.300d.txt')
for line300 in f:
  value300 = line300.split()
  word300 = value300[0]
  coef300 = np.asarray(value300[1:],dtype='float32')
  embedding_index_glove300[word300] = coef300

print("%d gefunden: "% len(embedding_index_glove300))


glove_matrix300 = np.zeros((vocabulary_size,300))
for word300, index300 in tokenizer.word_index.items():
    embedding_value300 = embedding_index_glove300.get(word300)
    if embedding_value300 is not None:
      glove_matrix300[index300] = embedding_value300
      hits300+=1
    else:
      misses300+=1

print("hits %d and %d misses for glove 300"%(hits300,misses300))


1917494 gefunden: 
hits 6963 and 8167 misses for glove 300


weiterer Test mit 50 Dim, sollte gleich sein wie zuvor

In [16]:
hits50 = 0
misses50 = 0

embedding_index_glove50 = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.twitter.27B.50d.txt')
for line50 in f:
  value50 = line50.split()
  word50 = value50[0]
  coef50 = np.asarray(value50[1:],dtype='float32')
  embedding_index_glove50[word50] = coef50

print("%d gefunden: "% len(embedding_index_glove50))


glove_matrix50 = np.zeros((vocabulary_size,50))
for word50, index50 in tokenizer.word_index.items():
    embedding_value50 = embedding_index_glove50.get(word50)
    if embedding_value50 is not None:
      glove_matrix50[index50] = embedding_value50
      hits50+=1
    else:
      misses50+=1

print("hits %d and %d misses for glove 50 "%(hits50,misses50))


1193514 gefunden: 
hits 6747 and 8383 misses for glove 50 


und noch mit glove 6B (200d, gibt auch andere Dimensions)

In [17]:

hits6b = 0
misses6b = 0

vocabulary_size = len(word_index)+2

embedding_index_glove6b = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.6B.200d.txt')
for line6b in f:
  value6b = line6b.split()
  word6b = value6b[0]
  coef6b = np.asarray(value6b[1:],dtype='float32')
  embedding_index_glove6b[word6b] = coef6b

print("%d gefunden: "% len(embedding_index_glove6b))


glove_matrix6b = np.zeros((vocabulary_size,200))
for word6b, index6b in tokenizer.word_index.items():
    embedding_value6b = embedding_index_glove6b.get(word6b)
    if embedding_value6b is not None:
      glove_matrix6b[index6b] = embedding_value6b
      hits6b+=1
    else:
      misses6b+=1

print("hits %d and %d misses for glove 6b"%(hits6b,misses6b))


400000 gefunden: 
hits 3721 and 11409 misses for glove 6b
